# Install libraries needed

In [1]:
!pip install pandas-gbq
!pip install google-cloud-bigquery
!pip install google-cloud-bigquery-storage

#Needed to setup flex slots for flat-rate pricing
!pip install google-cloud-bigquery-reservation

# Automatically restart kernel after installs
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)


     |████████████████████████████████| 91 kB 6.2 MB/s eta 0:00:011


{'status': 'ok', 'restart': True}

## Import packages

Here we will install the packages needed for the project as follows:

In [2]:
from google.cloud import bigquery
import numpy as np
import pandas as pd
import pandas_gbq
import matplotlib.pyplot as plt

# Used to display float format
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Create a BigQuery Client

Create a Bigquery client using the python client library for BigQuery

In [3]:
client = bigquery.Client(project = 'bigquery-kmeans')

## Process the data

Here, we will prepare the data we need to train the k-means model. Preparing the data includes aggregating customer transaction data, generating sample customer demographic data, and then joining these two datasets to create the training dataset

### Create a BigQuery dataset
Here I will create a BigQuery dataset to contain the customer data and the k-means model

In [8]:
! bq mk bigquery-kmeans:kmeans
    

Dataset 'bigquery-kmeans:kmeans' successfully created.


### Aggregate transaction data
Next, we aggregate customer transaction data from the sample analytics 360 dataset. The aggregated data should provide us with a cleare picture of how often customer purchase items, what types of items they purchase, and how much they spend

For our purpose, we assume that a purchase occurs when the pafetype = 'EVENT'. In a production use case, you might want to identify purchases in some other way, depending on your configuration of analytics 360

So, we will use the ```fullVisistorID``` as the key for the customer data. In a production use case, you should replace instances off ```fullVisitorID``` with ```clientID``` in this and subsequent queries. This is because you must use ```clientID``` as the key field when importing audience data into analytics 360. ```clientID``` is normally a hashed version of ```fullVisitorID``` but this column isnt populated in the sample dataset. 

1. Create a view with aggregated customer transaction data

In [15]:
ga360_only_view = 'GA360_View'
shared_dataset_ref = client.dataset('kmeans')
ga360_view_ref = shared_dataset_ref.table(ga360_only_view)
ga360_view = bigquery.Table(ga360_view_ref)


In [16]:
ga360_query = '''
SELECT
  fullVisitorID,
  # This will be used to generate random data.
  ABS(farm_fingerprint(fullVisitorID)) AS Hashed_fullVisitorID,
  # You can aggregate this because an
  # operating system is tied to a fullVisitorID.
  MAX(device.operatingSystem) AS OS,
  SUM (CASE
       WHEN REGEXP_EXTRACT (v2ProductCategory,
                           r'^(?:(?:.*?)Home/)(.*?)/')
                           = 'Apparel' THEN 1 ELSE 0 END) AS Apparel,
  SUM (CASE
       WHEN REGEXP_EXTRACT (v2ProductCategory,
                           r'^(?:(?:.*?)Home/)(.*?)/')
                           = 'Office' THEN 1 ELSE 0 END) AS Office,
  SUM (CASE
       WHEN REGEXP_EXTRACT (v2ProductCategory,
                           r'^(?:(?:.*?)Home/)(.*?)/')
                           = 'Electronics' THEN 1 ELSE 0 END) AS Electronics,
  SUM (CASE
       WHEN REGEXP_EXTRACT (v2ProductCategory,
                           r'^(?:(?:.*?)Home/)(.*?)/')
                           = 'Limited Supply' THEN 1 ELSE 0 END) AS LimitedSupply,
  SUM (CASE
       WHEN REGEXP_EXTRACT (v2ProductCategory,
                           r'^(?:(?:.*?)Home/)(.*?)/')
                           = 'Accessories' THEN 1 ELSE 0 END) AS Accessories,
  SUM (CASE
       WHEN REGEXP_EXTRACT (v2ProductCategory,
                           r'^(?:(?:.*?)Home/)(.*?)/')
                           = 'Shop by Brand' THEN 1 ELSE 0 END) AS ShopByBrand,
  SUM (CASE
       WHEN REGEXP_EXTRACT (v2ProductCategory,
                           r'^(?:(?:.*?)Home/)(.*?)/')
                           = 'Bags' THEN 1 ELSE 0 END) AS Bags,
  ROUND (SUM (productPrice/1000000),2) AS totalSpent_USD
  FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
  UNNEST(hits) AS hits,
  UNNEST(hits.product) AS hits_product
WHERE
  _TABLE_SUFFIX BETWEEN '20160801'
  AND '20160831'
  AND geoNetwork.country = 'United States'
  AND type = 'EVENT'
GROUP BY
  1,
  2
'''

In [17]:
ga360_view.view_query = ga360_query.format('bigquery-kmeans')
ga360_view = client.create_table(ga360_view)

Lets see a sample of the aggregated customer transaction data 

In [18]:
#sample ga360 data

ga360_query_df = f'''
SELECT * FROM {ga360_view.full_table_id.replace(":", ".")}
ORDER BY fullVisitorid
LIMIT 5
'''

job_config = bigquery.QueryJobConfig()

# start the query
query_job = client.query(ga360_query_df, job_config=job_config) #API request
df_ga360 = query_job.result()
df_ga360 = df_ga360.to_dataframe()

df_ga360

,fullVisitorID,Hashed_fullVisitorID,OS,Apparel,Office,Electronics,LimitedSupply,Accessories,ShopByBrand,Bags,totalSpent_USD
0,000005103959234087,4337061952752469967,Android,0,1,0,0,0,0,0,11.990
1,0003130024474361102,1959238978674290279,Chrome OS,0,22,0,0,0,0,0,68.890
2,000412118581900885,2335900684219890205,Windows,4,0,0,2,0,0,0,227.940
3,0004824403431530928,7205701007295976334,Macintosh,10,2,0,6,2,0,0,1044.820
4,0006056564841557616,8664318594051821921,Macintosh,2,0,0,0,0,0,0,33.980


### Generate demographic data

So we will generate synthetic customer demographic data

In a production use case, instead of generating fake data we could substitute this process with data from a CRM system.

In [19]:
# Create a view with generated customer data

CRM_only_view = 'CRM_View'
shared_dataset_ref = client.dataset('kmeans')
CRM_view_ref = shared_dataset_ref.table(CRM_only_view)
CRM_view = bigquery.Table(CRM_view_ref)


The query below works by hashing the fullVisitorID, which creates a random distribution. It uses modulo to artificially split gender and household income distribution

In [20]:
CRM_query = '''
SELECT
  fullVisitorID,
IF
  (MOD(Hashed_fullVisitorID,2) = 0,
    "M",
    "F") AS gender,
  # Generate household income
  CASE
    WHEN MOD(Hashed_fullVisitorID,10) = 0 THEN 55000
    WHEN MOD(Hashed_fullVisitorID,10) < 3 THEN 65000
    WHEN MOD(Hashed_fullVisitorID,10) < 7 THEN 75000
    WHEN MOD(Hashed_fullVisitorID,10) < 9 THEN 85000
    WHEN MOD(Hashed_fullVisitorID,10) = 9 THEN 95000
  ELSE
  Hashed_fullVisitorID
END
  AS hhi
FROM (
  SELECT
    fullVisitorID,
    ABS(farm_fingerprint(fullVisitorID)) AS Hashed_fullVisitorID,
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
    UNNEST(hits) AS hits,
    UNNEST(hits.product) AS hits_product
      WHERE
    _TABLE_SUFFIX BETWEEN '20160801'
    AND '20160831'
    AND geoNetwork.country = 'United States'
    AND type = 'EVENT'
  GROUP BY
    1,
    2)
'''

In [21]:
CRM_view.view_query = CRM_query.format('bigquery-kmeans')
CRM_view = client.create_table(CRM_view)

See a sample of the generated customer data

In [23]:
# See an output of the synthetic CRM data

CRM_query_df = f'''
SELECT * FROM {CRM_view.full_table_id.replace(":", ".")}
ORDER BY fullVisitorID
LIMIT 5
'''

job_config = bigquery.QueryJobConfig()

# Start the query
query_job = client.query(CRM_query_df, job_config=job_config)
df_CRM = query_job.result()
df_CRM = df_CRM.to_dataframe()

df_CRM

,fullVisitorID,gender,hhi
0,000005103959234087,F,85000
1,0003130024474361102,F,95000
2,000412118581900885,F,75000
3,0004824403431530928,M,75000
4,0006056564841557616,F,65000
